# 数据集快速入门
tf.data 模块包含一系列类，可让您轻松地加载数据、操作数据并通过管道将数据传送到模型中。本文档通过两个简单的示例来介绍该 API：

从 Numpy 数组中读取内存中的数据。

从 csv 文件中读取行

# 基本输入

In [1]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

## 参数
此函数需要三个参数。要求所赋值为“数组”的参数能够接受可通过 numpy.array 转换成数组的几乎任何值。其中存在一个例外，即对 Datasets 有特殊意义的 tuple，稍后我们会发现这一点。

features：包含原始输入特征的 {'feature_name':array} 字典（或 DataFrame）。

labels：包含每个样本的标签的数组。

batch_size：表示所需批次大小的整数。

## 切片
首先，此函数会利用 tf.data.Dataset.from_tensor_slices 函数创建一个代表数组切片的 tf.data.Dataset。系统会在第一个维度内对该数组进行切片。例如，一个包含 mnist 训练数据的数组的形状为 (60000, 28, 28)。将该数组传递给 from_tensor_slices 会返回一个包含 60000 个切片的 Dataset 对象，其中每个切片都是一个 28x28 的图像。

### MNIST示例

In [2]:
import tensorflow as tf
train,test = tf.keras.datasets.mnist.load_data() 
mnist_x,mnist_y = train

mnist_ds = tf.data.Dataset.from_tensor_slices(mnist_x)
print(mnist_ds)

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<TensorSliceDataset shapes: (28, 28), types: tf.uint8>


### 鸢尾花示例

In [3]:
#版权归https://www.tensorflow.org/get_started/get_started_for_beginners
from __future__ import  absolute_import,division,print_function

import os
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
#import tensorflow.contrib.eager as tfe
import argparse
#tf.enable_eager_execution()

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', default=100, type=int, help='batch size')
parser.add_argument('--train_steps', default=1000, type=int,
                    help='number of training steps')

print("tensorflow version:{}".format(tf.VERSION))
print("Eager Excution:{}".format(tf.executing_eagerly()))

tensorflow version:1.9.0
Eager Excution:False


In [4]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']

SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def load_data(label_name = 'Species'):
    #训练集提取
    train_path = tf.keras.utils.get_file(fname= os.path.basename(TRAIN_URL),
                                        origin = TRAIN_URL)
    train = pd.read_csv(filepath_or_buffer=train_path,names=CSV_COLUMN_NAMES,header=0)
   
    train_features,train_label = train,train.pop(label_name)
    
    #测试集提取
    test_path = tf.keras.utils.get_file(fname=os.path.basename(TEST_URL),
                                       origin = TEST_URL)
    test = pd.read_csv(filepath_or_buffer=test_path,names=CSV_COLUMN_NAMES,header=0)
    
    test_features,test_label = test,test.pop(label_name)
    
    return(train_features,train_label),(test_features,test_label)

In [5]:
(train_x,train_y),(test_x,test_y) = load_data()

In [6]:
dataset = tf.data.Dataset.from_tensor_slices(dict(train_x))
print(dataset)

<TensorSliceDataset shapes: {SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, types: {SepalLength: tf.float64, SepalWidth: tf.float64, PetalLength: tf.float64, PetalWidth: tf.float64}>


In [7]:
dataset = tf.data.Dataset.from_tensor_slices((dict(train_x), train_y))
print(dataset)

<TensorSliceDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float64, SepalWidth: tf.float64, PetalLength: tf.float64, PetalWidth: tf.float64}, tf.int64)>


## 操作
目前，Dataset 会按固定顺序迭代数据一次，并且一次仅生成一个元素。它需要进一步处理才可用于训练。幸运的是，tf.data.Dataset 类提供了更好地准备训练数据的方法

### shuffle、repeat、batch
shuffle 方法使用一个固定大小的缓冲区，在条目经过时随机化处理条目。在这种情况下，buffer_size 大于 Dataset 中样本的数量，确保数据完全被随机化处理.

repeat 方法会在结束时重启 Dataset。要限制周期数量，请设置 count 参数。

batch 方法会收集大量样本并将它们堆叠起来以创建批次。这为批次的形状增加了一个维度。

In [8]:
# Shuffle, repeat, and batch the examples.
dataset = dataset.shuffle(1000).repeat().batch(100)

In [9]:
print(mnist_ds.batch(100))

<BatchDataset shapes: (?, 28, 28), types: tf.uint8>


In [10]:
print(dataset)

<BatchDataset shapes: ({SepalLength: (?,), SepalWidth: (?,), PetalLength: (?,), PetalWidth: (?,)}, (?,)), types: ({SepalLength: tf.float64, SepalWidth: tf.float64, PetalLength: tf.float64, PetalWidth: tf.float64}, tf.int64)>


# 读取CSV文件

## 构建 Dataset
我们先构建一个 TextLineDataset 对象，实现一次读取文件中的一行数据。然后，我们调用 skip 方法来跳过文件的第一行，此行包含标题，而非样本

In [14]:
import iris_data
train_path,test_path = iris_data.maybe_download()
ds = tf.data.TextLineDataset(train_path).skip(1)

## 构建 csv 行解析器

In [22]:
# Metadata describing the text columns
COLUMNS = ['SepalLength', 'SepalWidth',
           'PetalLength', 'PetalWidth',
           'label']
FIELD_DEFAULTS = [[0.0],[0.0],[0.0],[0.0],[0]]
def _parse_line(line):
    #Decode the line into its fields
    fields = tf.decode_csv(line,FIELD_DEFAULTS)
    
    # Pack the result into a dictionary
    features = dict(zip(COLUMNS,fields))
    
    # Separate the label from the features
    label = features.pop('label')
    
    return features,label


### 解析行
数据集提供很多用于在通过管道将数据传送到模型的过程中处理数据的方法。最常用的方法是 map，它会对 Dataset 的每个元素应用转换。

map 方法会接受 map_func 参数，此参数描述了应该如何转换 Dataset 中的每个条目。

In [16]:
ds = ds.map(_parse_line)
print(ds)

<MapDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float32, SepalWidth: tf.float32, PetalLength: tf.float32, PetalWidth: tf.float32}, tf.int32)>


## 试试看
此函数可用于替换 iris_data.train_input_fn。可使用此函数馈送 Estimator，如下所示：

In [26]:
train_path, test_path = iris_data.maybe_download()

# All the inputs are numeric
feature_columns = [
    tf.feature_column.numeric_column(name)
    for name in iris_data.CSV_COLUMN_NAMES[:-1] 
]

#builid the estimator
est = tf.estimator.LinearClassifier(feature_columns,
                                    n_classes=3)

#Train the estimator
batch_size = 100
est.train(
    steps = 1000,
    input_fn = lambda:iris_data.csv_input_fn(train_path,batch_size))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\THINK\\AppData\\Local\\Temp\\tmpl_qc7fjr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000017C64E55630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 int